In [2]:
import numpy as np
import plotly.graph_objs as go 
from ipywidgets import widgets
 
import scipy.stats as spst

label1=widgets.Label('seed for proposal')

label2=widgets.Label('seed for acceptance/rejection')

    
seed_value1=widgets.IntSlider(
    value=5,
    min=0,
    max=20,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

seed_value2=widgets.IntSlider(
    value=5,
    min=0,
    max=20,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


steps=widgets.ToggleButtons(
   # value=False,
    #description='Click me',
   # disabled=False,
    options=["propose","accept/reject","final"],
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

N=1000

PMF_p=np.repeat(0.25,4)
PMF_t=np.array([0.1,0.6,0.1,0.2])
x=np.arange(1,5)

def proposal(seed_v1):
    ST1=np.random.RandomState(seed_v1+1000)
    samples=  (ST1.rand(N)*4).astype("int")+1
    return samples

def reject(samples,seed_v2):
    ST2=np.random.RandomState(seed_v2+2000)
    c=np.max(PMF_t/PMF_p)
    AR=PMF_t[samples-1]/PMF_p[samples-1]/c
    return ST2.rand(N)>AR
    
def AR_samples(seed_v1,seed_v2):     
    samples=proposal(seed_v1)
    rejected=reject(samples,seed_v2)
    np.random.seed(100)
    scatterX=samples+0.4-np.random.rand(N)*0.8  
    scatterY=PMF_t[samples-1]*np.random.rand(N)
    scatterX1=scatterX[rejected]  
    scatterY1=scatterY[rejected]  
    scatterX2=scatterX[~rejected]  
    scatterY2=scatterY[~rejected]  
    return scatterX1,scatterX2,scatterY1,scatterY2, scatterX,scatterY 
scatterX1,scatterX2,scatterY1,scatterY2,scatterX,scatterY =AR_samples(int(seed_value1.value),int(seed_value2.value))
 

trace1=go.Bar(x=x,y=PMF_t, 
              hoverinfo="text",text="",name="Target distribution"+' '*10  , width=0.8 )

trace2=go.Scatter(x=scatterX1,y=scatterY1,hoverinfo="text", text="",
                  marker=dict(size=5,color="red"),name="Rejected Samples",mode="markers" )
trace3=go.Scatter(x=scatterX2,y=scatterY2,hoverinfo="text",text="",
                  marker=dict(size=5,color="yellow"),name="Accepted samples",mode="markers" )
trace4=go.Scatter(x=scatterX,y=scatterY,hoverinfo="text",text="",
                  marker=dict(size=5,color="yellow"),name="Proposed Samples",mode="markers" )

g = go.FigureWidget(data=[trace1,trace4],
                        layout=go.Layout(
                        title=dict(
                        text='rejection sampling'
                        ),
                
                        xaxis = dict(title = " x values", range = [0.5,4.5],showticklabels=True),
                        yaxis = dict(title = "probability", range = [-.1,0.8]),
                        hovermode=None,
                        margin={'l': 0, 'r': 0, 't': 0, 'b': 0},width=1000, height=400  ),

                   )
g.update_layout(
                     title_x=0.5,
                 title_y=0.9,

 legend=dict(

                    x=0.7,
                    y=0.7,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ))
                   
)
        
def response2(change):
    scatterX1,scatterX2,scatterY1,scatterY2,scatterX,scatterY=AR_samples(int(seed_value1.value),int(seed_value2.value))
    if steps.value=="propose":
        with g.batch_update():
            g.data = [g.data[0]]
            trace4=go.Scatter(x=scatterX,y=scatterY,hoverinfo="text",text="",
                  marker=dict(size=5,color="yellow"),name="Proposed Samples",mode="markers" )
            g.add_trace(trace4)
    elif  steps.value=="accept/reject":
        with g.batch_update():
            g.data = [g.data[0]]
            trace2=go.Scatter(x=scatterX1,y=scatterY1,hoverinfo="text", text="",
                  marker=dict(size=5,color="red"),name="Rejected Samples",mode="markers" )
            trace3=go.Scatter(x=scatterX2,y=scatterY2,hoverinfo="text",text="",
                  marker=dict(size=5,color="yellow"),name="Accepted samples",mode="markers" )
            g.add_trace(trace2)
            g.add_trace(trace3)
 
            
    else:
        with g.batch_update():
            g.data = [g.data[0]]
 
            trace3=go.Scatter(x=scatterX2,y=scatterY2,hoverinfo="text",text="",
                  marker=dict(size=5,color="yellow"),name="Accepted samples",mode="markers" )    
            g.add_trace(trace3)
    
container11 = widgets.VBox([label1,label2]) 
container12 = widgets.VBox([seed_value1,seed_value2])     
container1 = widgets.HBox([container11,container12 ])
container2=widgets.HBox([steps ])
         
 
container3 = widgets.HBox([g ])

 
steps.observe(response2,names="value")
seed_value1.observe(response2,names="value")
seed_value2.observe(response2,names="value")


 

Widget1=widgets.VBox([container1,container2,container3] )
Widget1